# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import json

# an iPython  "magic" that enables the embedding of matplotlib output
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

with open("cases.json", "rt") as INFILE:
    data=json.load(INFILE)
    
datalist=data['data']

dates=[dictionary['date'] for dictionary in datalist ]
dates.sort()

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

startdate=parse_date(dates[0])
enddate=parse_date(dates[-1])

index=pd.date_range(startdate, enddate, freq='D')
casesdf=pd.DataFrame(index=index, columns=['dailyCases', 'cumulativeCases', 'dailyDeaths', 'cumulativeDeaths'])
casesdf

for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
    date=parse_date(entry['date'])
    for column in ['dailyCases', 'cumulativeCases', 'dailyDeaths', 'cumulativeDeaths']:
        if pd.isna(casesdf.loc[date, column]): 
            # replace None with 0 in our data 
            value= float(entry[column]) if entry[column]!=None else 0.0
            # this is the way you access a specific location in the dataframe - use .loc
            # and put index,column in a single set of [ ]
            casesdf.loc[date, column]=value
            
# fill in any remaining "holes" due to missing dates
#casesdf.fillna(0.0, inplace=True) --> this prevents the data which goes back to zero from filling the graph completely
            
casesdf

,dailyCases,cumulativeCases,dailyDeaths,cumulativeDeaths
2020-01-31,2.0,2.0,0.0,0.0
2020-02-01,0.0,2.0,0.0,0.0
2020-02-02,0.0,2.0,0.0,0.0
2020-02-03,0.0,2.0,0.0,0.0
2020-02-04,0.0,2.0,0.0,0.0
...,...,...,...,...
2023-10-15,0.0,20970982.0,0.0,0.0
2023-10-16,0.0,20970982.0,0.0,0.0
2023-10-17,0.0,20970982.0,0.0,0.0
2023-10-18,0.0,20970982.0,0.0,0.0


# Download current data & Graphs and Analysis

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [16]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

def access_api(button):
    apibutton.icon="check"
    apibutton.disabled=True
    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)

# register the callback function with the button
apibutton.on_click(access_api)

# this is an iPython function that generalises print for Jupyter Notebooks; we use it to 
# display the widgets
display(apibutton)

casesdf=pd.read_pickle("casesdf.pkl")

casescols=wdg.SelectMultiple(
    options=['dailyCases', 'cumulativeCases', 'dailyDeaths', 'cumulativeDeaths'],
    value=['dailyCases', 'cumulativeCases', 'dailyDeaths', 'cumulativeDeaths'],
    rows=4,
    description="Cases: ",
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
#   value='pineapple', # Defaults to 'pineapple'
#   layout={'width': 'max-content'}, # If the items' names are long
    description="Scale: ",
    disabled=False
)

# try replacing HBox with a VBox
controls=wdg.HBox([scale, casescols])

def cases_graphs(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        casesdf[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
# capture output in widget graph   
graph=wdg.interactive_output(cases_graphs, {'gcols': casescols, 'gscale': scale})

display(controls, graph)


Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

Output()